# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,malango,-2.9627,119.9001,27.89,62,45,1.14,ID,1712802009
1,1,west island,-12.1568,96.8225,27.99,69,40,5.14,CC,1712801546
2,2,stanley,54.8680,-1.6985,13.01,87,40,7.72,GB,1712801550
3,3,thompson,55.7435,-97.8558,0.58,99,100,2.46,CA,1712801556
4,4,adamstown,-25.0660,-130.1015,21.89,88,100,10.32,PN,1712801547


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
import warnings
import holoviews as hv
from holoviews.element.tiles import OSM

# Suppress the CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",             # Adjust column name for longitude if necessary
    "Lat",             # Adjust column name for latitude if necessary
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",   # Size of the point represents humidity
    color="City",      # Use 'City' column for colors
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7          # Set transparency to 0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_humidity



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Filter cities based on criteria (e.g., temperature between 20°C and 25°C, humidity below 50%, cloudiness below 30%, wind speed below 5 m/s)
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 10) & (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 50) & 
                                  (city_data_df['Wind Speed'] < 5)].copy()

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,aberglasslyn,-32.7000,151.5333,22.57,47,28,2.38,AU,1712802014
65,65,torrox,36.7579,-3.9523,14.25,49,15,1.79,ES,1712802015
108,108,dongola,19.1667,30.4833,20.90,18,2,4.83,SD,1712802019
109,109,parkes,-33.1333,148.1833,17.97,45,0,2.37,AU,1712802019
116,116,ouarzazat,30.9189,-6.8934,19.16,8,0,2.57,MA,1712802020


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
58,aberglasslyn,AU,-32.7000,151.5333,47,
65,torrox,ES,36.7579,-3.9523,49,
108,dongola,SD,19.1667,30.4833,18,
109,parkes,AU,-33.1333,148.1833,45,
116,ouarzazat,MA,30.9189,-6.8934,8,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:

# Set parameters to search for a hotel
radius = 10000  # Define the search radius in meters
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df.head()

Starting hotel search
aberglasslyn - nearest hotel: Maitland Monte Pio
torrox - nearest hotel: Hotel "La CASA"
dongola - nearest hotel: Lord
parkes - nearest hotel: Commercial Hotel
ouarzazat - nearest hotel: Fint
ulanhot - nearest hotel: 阿勒泰大酒店
nenjiang - nearest hotel: No hotel found
abashiri - nearest hotel: ホテル オホーツク・イン
mhamid - nearest hotel: Hotel Kasbah Azalay
forbes - nearest hotel: No hotel found
kerman - nearest hotel: حافظ
al muwayh - nearest hotel: فندق ومطعم الموية
seryshevo - nearest hotel: Гостиница &quot;Тополек&quot;
yulinshi - nearest hotel: 榆溪大酒店
liman - nearest hotel: Зоря
kharan - nearest hotel: No hotel found
beyneu - nearest hotel: Johan yu
tlahualilo de zaragoza - nearest hotel: No hotel found
qalat - nearest hotel: No hotel found
kawm umbu - nearest hotel: No hotel found
xining - nearest hotel: Qinghai hotel
calhoun - nearest hotel: Quality Inn
turabah - nearest hotel: No hotel found
ascencion - nearest hotel: Hotel Miami
sultanah - nearest hotel: فندق جلنار
pi

,City,Country,Lat,Lng,Humidity,Hotel Name
58,aberglasslyn,AU,-32.7000,151.5333,47,Maitland Monte Pio
65,torrox,ES,36.7579,-3.9523,49,"Hotel ""La CASA"""
108,dongola,SD,19.1667,30.4833,18,Lord
109,parkes,AU,-33.1333,148.1833,45,Commercial Hotel
116,ouarzazat,MA,30.9189,-6.8934,8,Fint


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Configure the map plot with hotel names and country information in the hover message
map_with_hotel_info = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],  # Add hotel name and country to hover message
    color="City",
    cmap='Category10', # Choose a colormap with distinct colors for categories
    alpha=0.7
).opts(
    xlabel="Longitude",
    ylabel="Latitude"
)

# Display the map plot
map_with_hotel_info


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)